In [1]:
#http://www.superrugby.co.nz/Grandstand/HistoricalStandings/30

# import libraries
import requests
from bs4 import BeautifulSoup

# specify the url
url = 'http://www.superrugby.co.nz/Grandstand/HistoricalResults/'
page = requests.get(url + '2005') # 2005 Super 12
# need to get years 2006 - 2018

# Create a BeautifulSoup object
soup = BeautifulSoup(page.text, 'html.parser').find("div", {"class": "content"})

In [2]:
season = soup.h2
print(season)

<h2>2005 Super 12</h2>


In [7]:
data = soup.table

In [40]:
import pandas as pd

table_nice = []
table_body = data.find('tbody')

rows = table_body.find_all('tr')
for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    search = 'Round'
    for col in cols:
        if col.startswith(search):
            #print('Found it!' + col)
            #print(col)
            break        
    table_nice.append([ele for ele in cols if ele]) # Get rid of empty values
    
print(type(table_nice))
df = pd.DataFrame(table_nice)
print(df.head(5))

<class 'list'>
         0                     1          2        3      4
0  Round 1                  None       None     None   None
1   25 Feb   Highlanders v Blues    Dunedin  7:35 PM  14-30
2   25 Feb     Waratahs v Chiefs     Sydney  9:40 PM   25-7
3   26 Feb     Stormers v Sharks  Cape Town  5:10 AM  26-12
4   26 Feb  Brumbies v Crusaders   Canberra  9:00 PM  32-21


In [41]:
df.dropna(inplace = True) 
#data[1]= data[1].str.split("v", n = 1, expand = True) 
df.columns = ['date','teams','location',
                     'time','score']

def outcome(f):
    if f > 0:
        return 'W'
    elif f < 0:
        return 'L'
    else:
        return 'D'

df['home'] = df['teams'].str.split('v').str[0]
df['away'] = df['teams'].str.split('v').str[1]
df['hp'] = df['score'].str.split('-').str[0].astype('int') # home points
df['ap'] = df['score'].str.split('-').str[1].astype('int') # away points
df['sm'] = df['hp'] - df['ap'] # score margin
df['ho'] = [outcome(x) for x in df['sm']] # home outcome

remove_columns = ['teams','score']
df = df.drop(columns=remove_columns)

print(df.head(5))
print(df.columns)

     date   location      time          home         away  hp  ap  sm ho
1  25 Feb    Dunedin   7:35 PM  Highlanders         Blues  14  30 -16  L
2  25 Feb     Sydney   9:40 PM     Waratahs        Chiefs  25   7  18  W
3  26 Feb  Cape Town   5:10 AM     Stormers        Sharks  26  12  14  W
4  26 Feb   Canberra   9:00 PM     Brumbies     Crusaders  32  21  11  W
5  26 Feb   Brisbane  11:05 PM         Reds    Hurricanes  10  24 -14  L
Index(['date', 'location', 'time', 'home', 'away', 'hp', 'ap', 'sm', 'ho'], dtype='object')
